Az adathalmazt a [huggingface.com](https://huggingface.co/datasets/princeton-nlp/SWE-bench_Lite)-ról lehet letölteni. Az adathalma érdekesebb mezői és jeletésük lejebb le van írva. A többi mező a kiértékeléshez szükségesek, mint például a repositoryban milyen teszteket kell megoldania az ágensnek, illetve milyen korábbi tesztek működnek implicit, amelyeknek továbbra is működniük kéne.

- **instance_id:** Egyedi azonosító, ami a github repo tulajdonosából, repo nevéből és PR számból epül fel.
- **base_commit:** Az adott feladat előtti utolsó commit. Ezt kell lefuttani, hogy az eredeti állapotot elérjük
- **patch:** Az a változtatás a base_commit után (kivétel a testel kapcsolatos kód), ami megoldotta az adott problémát
- **problem_statement:** A feladat szöveges leírása

A feladat az, hogy ennek az adathalmaznak következtében egy JSON objectkumokat tartalmazó .jsonl file-t kell létrehozni aminek minden eleme a következő mezőkkel rendelkezik:

- **instance_id:** az adathalmaz egy rekordjának kulcsa, amivel azonosítani lehet melyik feladatra adtunk megoldást.
- **model:** az ágens által használt model neve. Csak dokumentáció érdekében van, mert ekkor már a megoldást megadjuk
- **prediction:** A diff patch string formályában. Az ágensek által generált változtatások, amik a feladatok megoldják.

In [1]:
from datasets import load_dataset
dataset = load_dataset('princeton-nlp/SWE-bench_Lite', split='test')

# Create a pandas dataframe
import pandas as pd
swebench = pd.DataFrame(dataset)

import platform
if platform.system() == "Windows":
    swebench['repo'] = swebench['repo'].apply(lambda x: x.replace('/', '\\'))

# Inspect the dataset
swebench.head(10)

D:\Projects\code-assistant-api\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,repo,instance_id,base_commit,patch,test_patch,problem_statement,hints_text,created_at,version,FAIL_TO_PASS,PASS_TO_PASS,environment_setup_commit
0,astropy\astropy,astropy__astropy-12907,d16bfe05a744909de4b27f5875fe0d4ed41ce607,diff --git a/astropy/modeling/separable.py b/a...,diff --git a/astropy/modeling/tests/test_separ...,Modeling's `separability_matrix` does not comp...,,2022-03-03T15:14:54Z,4.3,"[""astropy/modeling/tests/test_separable.py::te...","[""astropy/modeling/tests/test_separable.py::te...",298ccb478e6bf092953bca67a3d29dc6c35f6752
1,astropy\astropy,astropy__astropy-14182,a5917978be39d13cd90b517e1de4e7a539ffaa48,diff --git a/astropy/io/ascii/rst.py b/astropy...,diff --git a/astropy/io/ascii/tests/test_rst.p...,Please support header rows in RestructuredText...,,2022-12-16T11:13:37Z,5.1,"[""astropy/io/ascii/tests/test_rst.py::test_rst...","[""astropy/io/ascii/tests/test_rst.py::test_rea...",5f74eacbcc7fff707a44d8eb58adaa514cb7dcb5
2,astropy\astropy,astropy__astropy-14365,7269fa3e33e8d02485a647da91a5a2a60a06af61,diff --git a/astropy/io/ascii/qdp.py b/astropy...,diff --git a/astropy/io/ascii/tests/test_qdp.p...,ascii.qdp Table format assumes QDP commands ar...,Welcome to Astropy 👋 and thank you for your fi...,2023-02-06T19:20:34Z,5.1,"[""astropy/io/ascii/tests/test_qdp.py::test_rou...","[""astropy/io/ascii/tests/test_qdp.py::test_get...",5f74eacbcc7fff707a44d8eb58adaa514cb7dcb5
3,astropy\astropy,astropy__astropy-14995,b16c7d12ccbc7b2d20364b89fb44285bcbfede54,diff --git a/astropy/nddata/mixins/ndarithmeti...,diff --git a/astropy/nddata/mixins/tests/test_...,"In v5.3, NDDataRef mask propagation fails when...",Welcome to Astropy 👋 and thank you for your fi...,2023-06-27T19:48:18Z,5.2,"[""astropy/nddata/mixins/tests/test_ndarithmeti...","[""astropy/nddata/mixins/tests/test_ndarithmeti...",362f6df12abf9bd769d4915fabf955c993ea22cf
4,astropy\astropy,astropy__astropy-6938,c76af9ed6bb89bfba45b9f5bc1e635188278e2fa,diff --git a/astropy/io/fits/fitsrec.py b/astr...,diff --git a/astropy/io/fits/tests/test_checks...,Possible bug in io.fits related to D exponents...,It is tested with `astropy/io/fits/tests/test_...,2017-12-07T00:01:14Z,1.3,"[""astropy/io/fits/tests/test_checksum.py::Test...","[""astropy/io/fits/tests/test_checksum.py::Test...",848c8fa21332abd66b44efe3cb48b72377fb32cc
5,astropy\astropy,astropy__astropy-7746,d5bd3f68bb6d5ce3a61bdce9883ee750d1afade5,diff --git a/astropy/wcs/wcs.py b/astropy/wcs/...,diff --git a/astropy/wcs/tests/test_wcs.py b/a...,Issue when passing empty lists/arrays to WCS t...,,2018-08-20T14:07:20Z,1.3,"[""astropy/wcs/tests/test_wcs.py::test_zero_siz...","[""astropy/wcs/tests/test_wcs.py::TestMaps::tes...",848c8fa21332abd66b44efe3cb48b72377fb32cc
6,django\django,django__django-10914,e7fd69d051eaa67cb17f172a39b57253e9cb831a,diff --git a/django/conf/global_settings.py b/...,diff --git a/tests/test_utils/tests.py b/tests...,Set default FILE_UPLOAD_PERMISSION to 0o644.\n...,I think you're talking about ef70af77ec53160d5...,2019-01-30T13:13:20Z,3.0,"[""test_override_file_upload_permissions (test_...","[""test_allowed_database_chunked_cursor_queries...",419a78300f7cd27611196e1e464d50fd0385ff27
7,django\django,django__django-10924,bceadd2788dc2dad53eba0caae172bd8522fd483,diff --git a/django/db/models/fields/__init__....,diff --git a/tests/model_fields/test_filepathf...,Allow FilePathField path to accept a callable....,"So, to clarify, what exactly is the bug/featur...",2019-02-03T11:30:12Z,3.0,"[""test_callable_path (model_fields.test_filepa...","[""test_path (model_fields.test_filepathfield.F...",419a78300f7cd27611196e1e464d50fd0385ff27
8,django\django,django__django-11001,ef082ebb84f00e38af4e8880d04e8365c2766d34,diff --git a/django/db/models/sql/compiler.py ...,diff --git a/tests/expressions/tests.py b/test...,Incorrect removal of order_by clause created a...,Is there a reason you can't use ​conditional e...,2019-02-17T13:02:09Z,3.0,"[""test_order_by_multiline_sql (expressions.tes...","[""test_deconstruc

Ennek következtében definiáltam két funkciót. Első sorban a repository clone-ozására van szükség, hogy elérhető legyen a projekt kódja az ágenseknek. Opcionálisan implementáltam egy olyan függvényt, amit eltávolítja a projektet, amikor megfejtettük a feladat megoldását. Továbbá már nem lesz rá szükség.

Előfordulhat, hogy bizonyos felhasználói módban való indításkor nincs elérési engedéje az interpreternek az adott könyvtárhoz (ez káros kód elleni védekezés miatt adóthat). Ekkor a megfelelő engedélyekért a **enable_wrinting** függvény felelős. Továbbá nincs olyan *python* függvény, ami egy könyvtárat és nem üres alkönyvtárakat kitöröl, ezért sajátot definiáltam **delete_dir** néven.

In [2]:
import git

def clone_repo(name: str, commit: str, path: str, logger = None):
    # Derive url from name
    t_start = time.time()
    url = f'https://github.com/{name}.git'

    try:
        # Clone the repository without checking out the HEAD
        repository = git.Repo.clone_from(url, path, no_checkout=True)

        # Checkout to specific commit
        repository.git.checkout(commit)
    except Exception as e:
        print(e)
        return None
    t_end = time.time()

    if logger:
        # Log repository cloning
        inference_time = t_end - t_start
        logger.log('cloning_time', inference_time)

    return repository

In [9]:
import shutil
import stat

def handle_remove_readonly(path: str):
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            filename = os.path.join(root, name)
            os.chmod(filename, stat.S_IWRITE)

def delete_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path, onerror=handle_remove_readonly)

A **run_agent** függvény wrapper az ágens futtatásához (tudom, hogy szigorúan véve a wrap programozási minta mást jelent). A wrapper a logolását teszi lehetővé pár adatnak, mint például a tokenek száma, és az átlagos idő. A végén diagramokat készítek ezek eloszlásáról.

Az ágens üzeneteinek logolásához [OpenTelemetry](https://opentelemetry.io/) szabványt használok (pontosabban a smolagents használ), mely az LLM felé irányuló HTTP hívásokat szabványosítja. Ezt [Phoenix](https://phoenix.arize.com/)-el kiegészítve, láthatóvá válik az LLM és az ágensek közötti kommunikáció.

A token számot sajnos nem lehet egyszerűen megszerezni a smolagents framework-kel, hogy a csv file-ba tudjam loggolni. Ezért a **tiktoken** package-et használom, mely beszélgetésből kiszámítja a felhasznált tokeneket, egy adott modelnek megfelelően.

In [4]:
import tiktoken

def run_agent(agent, repo, prompt, logger = None):
    # Solve problem with Agent
    t_start = time.time()
    final_answer = agent.run(prompt)
    diff = repo.git.diff()
    t_end = time.time()

    # Log agent run
    if logger:
        inference_time = t_end - t_start
        logger.log('total_steps', len(agent.memory.get_full_steps()))
        logger.log("inference_time", inference_time)

        # Calculate tokens using a specific model
        encoding = tiktoken.encoding_for_model('gpt-4o-mini')
        logger.log('output_tokens', agent.monitor.total_output_token_count)
        logger.log('input_tokens', agent.monitor.total_input_token_count)

    return diff, final_answer

Ezt csak egyszer kell futtatni. Érdekességként, lehet instrumentációt használni ahhoz, hogy a kódot megspóroljuk. Ekkor alapértelmezett beállításokkal indítja el a szolgáltatást. Én viszont a manuális instrumentáció mellett döntöttem, mivel a **tracer** objektum segítségével lehet a futásokat elnevezni.

In [5]:
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from openinference.instrumentation import using_metadata

register(
    project_name="LLMCodeAct-Lite",
    endpoint="http://localhost:6006/v1/traces",
    auto_instrument=True # See 'Trace all calls made to a library' below
)
SmolagentsInstrumentor().instrument()
metadata = {
    'instance_id': "default",
    'attempt_nr': 0,
    'model': 'gpt-4o-mini'
}

Attempting to instrument while already instrumented


OpenTelemetry Tracing Details
|  Phoenix Project: LLMCodeAct-Lite
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



A **benchmark_repo** függvény, egyetlen instance_id benchmarkolását foglalja össze. A következő lépéseket definiálja:

- A repository clonozása
- Az ágens munkafolyamatának kontextusát átállítja a repository gyökerére. Ezen belül értelmesek a definiált eszközöt (tool-ok). Ezen belül tud navigálni, kilépni nem tud.
- Ha a repository clonozása valamilyen októl fogva sikertelen, akkor átlépi a repository-t ahhelyett, hogy hibával megállna.
- Az ágenst futtatja az adott probléma leírásával.
- A változtatásokból egy **diff** patchet készít. Ha nem voltak változtatások, akkor a diff üres marad. Ekkor újra futtatja az ágenst.
- Eltávolítja a repository-t. Csak az után lehet, miután a patchet kiolvastuk.s

Legnagyobb nehézséget az ágens makacssága jelenti, aminek köszönhetően nem akarja a projektet módosítani, hanem csak egy absztrakt leírást ad válaszként. Ezt orvosolja a **rethink** struktúra, mely megfigyeli, hogy változott-e a kód. Ha nem akkor újra futtatja az ágenst a legutobbi válasszal és egy új promptal, mely felhívja a figyelmét arra, hogy változtassa meg a kódot.

Itt nagyon jó lenne, ha új futtatás esetén, a korábbi futtatás egész csevegését csatolnánk az ágensre, így nincs szüksége újra megvizsgálnia a projektet.

In [10]:
from git import Repo
import time

def solve_task(
        agent, context, repo_name: str, instance_id:str,
        base_commit: str, problem_statement: str, logger = None
):
    t_start = time.time()
    local_path = os.path.join(os.getcwd(), repo_name)

    if not os.path.exists(local_path):
        # Clone the repository
        repo = clone_repo(
            name=repo_name,
            commit=base_commit,
            path=local_path,
            logger=logger
        )
    else:
        repo = Repo(local_path)
        logger.log('cloning_time', time.time() - t_start)

    # Set the context to the new repository
    # Check context manager
    context.set_root(local_path)
    assert context.get_root() != os.getcwd(), \
        f'Context is: {context.get_root()}'
    assert context.get(abs=True) == local_path, \
        f"Context management issue: {context.get(abs=True)} != {local_path}"
    assert context.get() == '.',\
        f"Context management issue: cwd is not root ({context.get()})"

    if repo is None:
        # In case of cloning issues skip this repository.
        raise RuntimeError(f'Repository {repo} cloning unsuccessful')

    # Generate patch file with agent
    attempt_number = 1
    metadata['attempt_nr'] = attempt_number
    metadata['instance_id'] = instance_id
    with using_metadata(metadata):
        diff, final_answer = run_agent(agent, repo, problem_statement, logger)

    # Agents sometime only solve problems in abstract term
    # Force agent to properly change the code.
    assert diff is not None, "Diff is none"
    while diff == '' and attempt_number <= 3 :
        attempt_number += 1
        message = (
            "\n\n It seams that you didn't make any changes to the project. "
            "You should solve the github issue, with directly inserting the "
            "python code in according files. "
            "You should also test your code by using the run_python tool."
        )
        metadata['attempt_nr'] = attempt_number
        metadata['instance_id'] = instance_id
        with using_metadata(metadata):
            prompt = problem_statement +  final_answer + message
            diff, final_answer = run_agent(agent, repo, prompt, logger)

    # Create the json object
    prediction = {
        "instance_id": instance_id,
        "model": "gpt-4o-mini",
        "prediction": diff
    }
    # Delete repository
    delete_dir(local_path)
    t_end = time.time()

    if logger:
        total_time = t_end - t_start
        logger.log('total_time', total_time)
        logger.log('attempts', attempt_number)

    return prediction

Innen látható hogy a második fázis fogalmaival foglalkozok jelenlegi verzióban. Még a benchmark előtt egy fontos feature-t akarok implementálni, ami nagyban fogja a teljesítményt befojásolni:

Utánnaolvastam az [SWE-agent](https://arxiv.org/pdf/2405.15793) cikkben, hogy ha a Plan-then-Act alapelv szerint müködtetem az ágenseket, akkor nagyobb valószínűséggel végeznek változtatásokat a projekten. Az elv szerint az ágenst arra kényszeríti, hogy előbb egy gondolatot fogalmazzon meg, majd egy akciót végezzen el. Az ágenseknek visszafogottak OS változtatások szempontjából.

In [1]:
# Create the agent
import os
from dotenv import load_dotenv
from smolagents import ToolCallingAgent, LiteLLMModel
from agents.tools import toolset

# Import environment variables from .env file
load_dotenv('.env')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

# Chose LLM model for chat completions
model = LiteLLMModel(
    model_id="openai/gpt-4o-mini",
    api_key=OPENAI_API_KEY
)
# Create code agent
coding_agent = ToolCallingAgent(
    model=model,
    tools=toolset,
    add_base_tools=True,
    verbosity_level=0,
    max_steps=25,
    name='coding_agent',
    description=(
        "You're a handy agent that can access projects, inspect and analyze the code,"
        "and make changes in the project in order to fix an issue "
        "The python interpreter tool can't access the variables and packages in the project, "
        "but you have at your disposal any necessary tool to search, navigate, change and execute"
        "python or any necessary files. "
        "Your job involves more action than thinking.."
    ),
)

D:\Projects\code-assistant-api\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'CodingAgent' from 'agents' (consider renaming 'D:\\Projects\\code-assistant-api\\agents\\__init__.py' if it has the same name as a library you intended to import)

Lejebb látható az adathalmaz első problémájának megoldása. A megoldás során a következő fejlesztési pontok fogalmazódott meg:

A CodeAct csak bizonyos kódot tud futtatni, pontosabban olyat, ami csak alapértelmezett python függvényeket tartalmaz (numpy, pandas, random, stb). Például os package függvényeit nem tudja meghívni. Ez abból adódik, hogy a fejlesztők nem kívánták azt, hogy a rossz szándékú kód módosítsa az oprendszer fájljait. A futás során viszont látható, hogy, értelemszerűen, a CodeAct olyan kódot ír, amiben felhasználja a repository függvényeit annak érdekében, hogy előállítsa a specifikált hibát.

Ezt viszont a következő képpen tudjuk orvosolni: container-ben futtatjuk a kódot. Ezt a smolagent natívan támogattja. Viszont ennek két hátránya van:

- A multi-agent mechanizmus nincs támogatva. Ehhez, a specifikáció szerint, saját docker sandbox-ot kell definiálni. Erről több itt olvasható: [Secure Code Execution](https://huggingface.co/docs/smolagents/en/tutorials/secure_code_execution)

- A repository-t be kell fecskendezni a container-be hogy elérhető legyen az ágensnek.

In [11]:
import json
from utils import CSVLogger, ContextManager

# Create the jsonl file
with open('test.jsonl', 'w') as f:
    f.write('')
logger = CSVLogger('benchmark.csv')
context = ContextManager(os.getcwd())

for index, task in swebench.iterrows():
    prediction = solve_task(
        agent=coding_agent,
        context=context,
        repo_name=task['repo'],
        instance_id=task['instance_id'],
        base_commit=task['base_commit'],
        problem_statement=task['problem_statement'],
        logger=logger
    )
    # Check instance_id against solution
    assert prediction['instance_id'] == task['instance_id'], \
        (f"Missmatch in problem and solution identifiers: "
         f"{prediction['instance_id']} vs. {task['instance_id']}")

    #Save prediction to the jsonl file.
    with open('test.jsonl', 'a', encoding='utf-8') as f:
        json_line = json.dumps(prediction)
        f.write(json_line + '\n')

    # Create new record from logged information
    logger.log('id', task['instance_id'])
    logger.save()
    break

TypeError: handle_remove_readonly() takes 1 positional argument but 3 were given